In [2]:
# import necessary modules from keras
from keras.layers import Dense,Softmax
from keras.models import Sequential

# creates a generic neural network architecture
model = Sequential()

# hidden layer takes a pre-processed frame as input, and has 200 units
model.add(Dense(units=200,input_dim=80*80, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(units=20,activation='relu', kernel_initializer='glorot_uniform'))
# model.add(Dense(units=200,input_dim=200, activation='relu', kernel_initializer='glorot_uniform'))



# output layer
model.add(Dense(units=1, activation='sigmoid', kernel_initializer='RandomNormal'))
# model.add(Softmax())

# compile the model using traditional Machine Learning losses and optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
import numpy as np
import gym
from gym import wrappers

# gym initialization
env = gym.make("Pong-v0")
env = wrappers.Monitor(env, 'tmp/pong-base', force=True)
observation = env.reset()
prev_input = None

# Macros
UP_ACTION = 2
DOWN_ACTION = 3
STOP = 0
act_space = {0:2,1:3,2:0}
# Hyperparameters
gamma = 0.99

# initialization of variables used in the main loop
x_train, y_train, rewards = [],[],[]
reward_sum = 0
episode_nb = 0

In [ ]:
from karpathy import prepro, discount_rewards

# main loop
while (True):
    env.render()
    # preprocess the observation, set input as difference between images
    cur_input = prepro(observation)
    x = cur_input - prev_input if prev_input is not None else np.zeros(80 * 80)
    prev_input = cur_input
    
    # forward the policy network and sample action according to the proba distribution
    proba = model.predict(np.expand_dims(x, axis=1).T)
    action = UP_ACTION if np.random.uniform() < proba else DOWN_ACTION
    y = 1 if action == 2 else 0 # 0 and 1 are our labels

    # log the input and label to train later
    x_train.append(x)
    y_train.append(y)

    # do one step in our environment
    observation, reward, done, info = env.step(action)
    rewards.append(reward)
    reward_sum += reward
    
    # end of an episode
    if done:
        print('At the end of episode', episode_nb, 'the total reward was :', reward_sum)
        
        # increment episode number
        episode_nb += 1
        
        # training
        model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, sample_weight=discount_rewards(rewards, gamma))
                                                             
        # Reinitialization
        x_train, y_train, rewards = [],[],[]
        observation = env.reset()
        reward_sum = 0
        prev_input = None

# main loop
# while (True):
#     env.render()
#     # preprocess the observation, set input as difference between images
#     cur_input = prepro(observation)
#     x = cur_input - prev_input if prev_input is not None else np.zeros(80 * 80)
#     prev_input = cur_input
    
#     # forward the policy network and sample action according to the proba distribution
#     proba = model.predict(np.expand_dims(x, axis=1).T)
#     proba = np.squeeze(proba,axis=0)
#     act_label = np.argmax(proba)
    
# #     print(len(proba), proba[act_label])
#     act_label_min = np.argmin(proba)
#     action = 0
#     if np.random.uniform() < proba[act_label]:
#         action = act_space[act_label] 
#     else:
#         if np.random.uniform() < proba[act_label_min]/(np.sum(proba)-proba[act_label]):
#             action = act_space[act_label_min] 
#         else:
#             res = list(set(act_space)-set([act_label,act_label_min]))[0]
#             action = act_space[res] 
#     y = np.zeros(3)
#     y[act_label] = 1 # if action == 2 else 0 # 0 and 1 are our labels

#     # log the input and label to train later
#     x_train.append(x)
#     y_train.append(y)

#     # do one step in our environment
#     observation, reward, done, info = env.step(action)
#     rewards.append(reward)
#     reward_sum += reward
    
#     # end of an episode
#     if done:
#         print('At the end of episode', episode_nb, 'the total reward was :', reward_sum)
        
#         # increment episode number
#         episode_nb += 1
        
#         # training
#         model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, sample_weight=discount_rewards(rewards, gamma))
                                                             
#         # Reinitialization
#         x_train, y_train, rewards = [],[],[]
#         observation = env.reset()
#         reward_sum = 0
#         prev_input = None

At the end of episode 0 the total reward was : -21.0
40/40 [==============================] - 1s 6ms/step - loss: -2.0426e-04 - accuracy: 0.4952
At the end of episode 1 the total reward was : -18.0
56/56 [==============================] - 0s 4ms/step - loss: 7.3456e-04 - accuracy: 0.5008
At the end of episode 2 the total reward was : -21.0
35/35 [==============================] - 0s 5ms/step - loss: 8.5644e-04 - accuracy: 0.5101
At the end of episode 3 the total reward was : -19.0
47/47 [==============================] - 0s 3ms/step - loss: -7.2152e-04 - accuracy: 0.5241
At the end of episode 4 the total reward was : -20.0
49/49 [==============================] - 0s 3ms/step - loss: 8.0778e-05 - accuracy: 0.5633
At the end of episode 5 the total reward was : -20.0
40/40 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 0.5426
At the end of episode 6 the total reward was : -21.0
40/40 [==============================] - 0s 4ms/step - loss: -0.0030 - accuracy: 0.51

35/35 [==============================] - 0s 3ms/step - loss: 0.0209 - accuracy: 0.8053
At the end of episode 58 the total reward was : -21.0
35/35 [==============================] - 0s 4ms/step - loss: 0.0195 - accuracy: 0.8091
At the end of episode 59 the total reward was : -21.0
42/42 [==============================] - 0s 3ms/step - loss: 0.0262 - accuracy: 0.8115
At the end of episode 60 the total reward was : -21.0
37/37 [==============================] - 0s 4ms/step - loss: -0.0201 - accuracy: 0.7591
At the end of episode 61 the total reward was : -21.0
40/40 [==============================] - 0s 3ms/step - loss: 0.0101 - accuracy: 0.7866
At the end of episode 62 the total reward was : -21.0
35/35 [==============================] - 0s 4ms/step - loss: 0.0060 - accuracy: 0.7733
At the end of episode 63 the total reward was : -21.0
46/46 [==============================] - 0s 5ms/step - loss: -0.0146 - accuracy: 0.7866
At the end of episode 64 the total reward was : -21.0
35/35 [====

38/38 [==============================] - 0s 3ms/step - loss: -0.0056 - accuracy: 0.9756
At the end of episode 116 the total reward was : -21.0
40/40 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9733
At the end of episode 117 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0182 - accuracy: 0.9643
At the end of episode 118 the total reward was : -21.0
44/44 [==============================] - 0s 3ms/step - loss: 0.0069 - accuracy: 0.9616
At the end of episode 119 the total reward was : -21.0
38/38 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 0.9680
At the end of episode 120 the total reward was : -21.0
34/34 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 0.9724
At the end of episode 121 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0068 - accuracy: 0.9707
At the end of episode 122 the total reward was : -21.0
35/35

38/38 [==============================] - 0s 5ms/step - loss: -0.0045 - accuracy: 0.9519
At the end of episode 173 the total reward was : -21.0
37/37 [==============================] - 0s 5ms/step - loss: -0.0070 - accuracy: 0.9484
At the end of episode 174 the total reward was : -21.0
48/48 [==============================] - 0s 5ms/step - loss: -0.0133 - accuracy: 0.9591
At the end of episode 175 the total reward was : -21.0
37/37 [==============================] - 0s 5ms/step - loss: -0.0107 - accuracy: 0.9670
At the end of episode 176 the total reward was : -21.0
34/34 [==============================] - 0s 5ms/step - loss: -0.0071 - accuracy: 0.9770
At the end of episode 177 the total reward was : -21.0
40/40 [==============================] - 0s 3ms/step - loss: 0.0071 - accuracy: 0.9730
At the end of episode 178 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 7.8323e-04 - accuracy: 0.9737
At the end of episode 179 the total reward was : -21

39/39 [==============================] - 0s 3ms/step - loss: 0.0161 - accuracy: 0.9518
At the end of episode 230 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: 0.0542 - accuracy: 0.9567
At the end of episode 231 the total reward was : -20.0
54/54 [==============================] - 0s 3ms/step - loss: 0.0380 - accuracy: 0.9354
At the end of episode 232 the total reward was : -21.0
40/40 [==============================] - 0s 5ms/step - loss: -0.0043 - accuracy: 0.9451
At the end of episode 233 the total reward was : -21.0
35/35 [==============================] - 0s 4ms/step - loss: 0.0210 - accuracy: 0.9315
At the end of episode 234 the total reward was : -20.0
39/39 [==============================] - 1s 26ms/step - loss: 0.0029 - accuracy: 0.9537
At the end of episode 235 the total reward was : -21.0
60/60 [==============================] - 0s 5ms/step - loss: 0.0052 - accuracy: 0.9509
At the end of episode 236 the total reward was : -21.0
45/4

...

45/45 [==============================] - 0s 3ms/step - loss: -0.0173 - accuracy: 0.9880
At the end of episode 914 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: -0.0047 - accuracy: 0.9910
At the end of episode 915 the total reward was : -21.0
38/38 [==============================] - 0s 3ms/step - loss: 0.0106 - accuracy: 0.9924
At the end of episode 916 the total reward was : -21.0
38/38 [==============================] - 0s 3ms/step - loss: 0.0158 - accuracy: 0.9890
At the end of episode 917 the total reward was : -21.0
42/42 [==============================] - 0s 3ms/step - loss: -0.0043 - accuracy: 0.9842
At the end of episode 918 the total reward was : -21.0
37/37 [==============================] - 0s 4ms/step - loss: -0.0212 - accuracy: 0.9923
At the end of episode 919 the total reward was : -21.0
39/39 [==============================] - 0s 3ms/step - loss: 0.0056 - accuracy: 0.9896
At the end of episode 920 the total reward was : -20.0
39

37/37 [==============================] - 0s 3ms/step - loss: -0.0030 - accuracy: 0.9974
At the end of episode 971 the total reward was : -20.0
37/37 [==============================] - 0s 3ms/step - loss: 0.0077 - accuracy: 0.9922
At the end of episode 972 the total reward was : -21.0
44/44 [==============================] - 0s 4ms/step - loss: -0.0361 - accuracy: 0.9936
At the end of episode 973 the total reward was : -21.0
61/61 [==============================] - 0s 3ms/step - loss: -0.0241 - accuracy: 0.9797
At the end of episode 974 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0158 - accuracy: 0.9866
At the end of episode 975 the total reward was : -21.0
34/34 [==============================] - 0s 3ms/step - loss: -0.0062 - accuracy: 0.9926
At the end of episode 976 the total reward was : -21.0
38/38 [==============================] - 0s 3ms/step - loss: 0.0019 - accuracy: 0.9891
At the end of episode 977 the total reward was : -21.0
50

40/40 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 0.9984
At the end of episode 1028 the total reward was : -21.0
52/52 [==============================] - 0s 3ms/step - loss: -0.0040 - accuracy: 0.9976
At the end of episode 1029 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: -0.0026 - accuracy: 0.9966
At the end of episode 1030 the total reward was : -21.0
38/38 [==============================] - 0s 3ms/step - loss: -0.0041 - accuracy: 0.9975
At the end of episode 1031 the total reward was : -21.0
52/52 [==============================] - 0s 3ms/step - loss: 0.0048 - accuracy: 0.9940 
At the end of episode 1032 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.9973
At the end of episode 1033 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 7.4437e-05 - accuracy: 0.9982
At the end of episode 1034 the total reward was

37/37 [==============================] - 0s 3ms/step - loss: -9.0581e-04 - accuracy: 0.9941
At the end of episode 1085 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: -0.0052 - accuracy: 0.9949
At the end of episode 1086 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: -2.2633e-04 - accuracy: 1.0000
At the end of episode 1087 the total reward was : -21.0
46/46 [==============================] - 0s 3ms/step - loss: -0.0103 - accuracy: 0.9958
At the end of episode 1088 the total reward was : -21.0
40/40 [==============================] - 0s 3ms/step - loss: 0.0040 - accuracy: 0.9953
At the end of episode 1089 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 0.9955
At the end of episode 1090 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: -0.0198 - accuracy: 0.9957
At the end of episode 1091 the total rewar

35/35 [==============================] - 0s 3ms/step - loss: -0.0020 - accuracy: 0.9973
At the end of episode 1142 the total reward was : -20.0
60/60 [==============================] - 0s 3ms/step - loss: 0.0050 - accuracy: 0.9900
At the end of episode 1143 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: -5.7342e-04 - accuracy: 0.9974
At the end of episode 1144 the total reward was : -21.0
37/37 [==============================] - 0s 3ms/step - loss: -0.0239 - accuracy: 0.9941
At the end of episode 1145 the total reward was : -21.0
35/35 [==============================] - 0s 3ms/step - loss: 0.0247 - accuracy: 0.9936
At the end of episode 1146 the total reward was : -20.0
36/36 [==============================] - 0s 4ms/step - loss: -8.3291e-04 - accuracy: 0.9991
At the end of episode 1147 the total reward was : -21.0
40/40 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 0.9976
At the end of episode 1148 the total reward